In [1]:
import os, random, argparse, json, time
from tqdm import tqdm, trange

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from texttable import Texttable
from typing import Union, Tuple, Dict, List

import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

c:\ProgramData\anaconda3\envs\pytorch\Lib\site-packages\torch_geometric\typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not find module 'C:\ProgramData\anaconda3\envs\pytorch\Lib\site-packages\torch_scatter\_scatter_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\ProgramData\anaconda3\envs\pytorch\Lib\site-packages\torch_geometric\typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: Could not find module 'C:\ProgramData\anaconda3\envs\pytorch\Lib\site-packages\torch_spline_conv\_basis_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(
c:\ProgramData\anaconda3\envs\pytorch\Lib\site-packages\torch_geometric\typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disablin

# Parameter

In [2]:
def parse_args():
    parser = argparse.ArgumentParser()  # 参数解析器对象

    parser.add_argument('--seed', type=int, default=10, help='Random seed of the experiment')
    parser.add_argument('--exp_name', type=str, default='Exp', help='Name of the experiment')
    
    parser.add_argument('--num_step', type=int, default=8640, help='4*24*90=8640') # TODO 90天数据
    parser.add_argument('--batch_size', type=int, default=96, help='Size of the batch') 
    parser.add_argument('--train_ratio' , type=float, default=3/6, help='train set ratio') # √
    parser.add_argument('--val_ratio'   , type=float, default=1/6, help='validation set ratio') # √
    parser.add_argument('--test_ratio'  , type=float, default=2/6, help='test set ratio') # √

    parser.add_argument('--num_history', type=int, default=1, help='Number of historical time steps') # √
    parser.add_argument('--ChebyshevDegree', type=int, default=8, help='degree of KAN') 
    parser.add_argument('--FourierDegree', type=int, default=6, help='degree of KAN') 

    parser.add_argument('--num_nodes', type=int, default=24, help='Number of nodes in the graph') # √
    parser.add_argument('--num_edges', type=int, default=34, help='Number of edges in the graph') # √
    parser.add_argument('--hidden_dim', type=int, default=12, help='Dimension of the hidden layers') # √
    
    parser.add_argument('--max_epoch', type=int, default=50, help='Maximum number of epochs') 
    parser.add_argument('--learning_rate', type=float, default=2e-5, help='Learning rate of Adam') 
    
    # L1正则化：鼓励模型选择较少的特征，从而实现特征选择。可以将某些权重压缩到零，意味着某些特征可能完全被忽略，由此简化模型。
    parser.add_argument('--l1_lambda', type=float, default=0.00, help='L1 regularization coefficient') # ×
    # L2正则化：鼓励模型的权重较小，从而使模型更简单。但并不会将它们压缩到零，从而保留了所有特征，避免过拟合。
    parser.add_argument('--l2_lambda', type=float, default=0.00, help='L2 regularization coefficient') # ×

    # parser.add_argument('--max_norm', type=float, default=0.30, help='梯度裁剪') # TODO 

    parser.add_argument('--gamma', type=float, default=0.95, help='decay parameter') 
    parser.add_argument('--decay_epoch', type=float, default=1, help='decay epoch') 
    parser.add_argument('--patience', type=int, default=10, help='提前停止的耐心参数') 

    parser.add_argument('--pressure_sensor_name', type=str, default='["4", "13", "14", "23", "24"]', help='压力传感器名称')
    parser.add_argument('--flow_sensor_name', type=str, default='["4", "7", "25", "28", "34"]', help='流量传感器名称')
    parser.add_argument('--demand_sensor_name', type=str, default='["2", "16", "19", "22", "24"]', help='需水量传感器名称')
    
    parser.add_argument('--water_pressure_file', type=str, default="D:/GraphormerForRobustness/dataset2/demand_patten_3612/simulate_pressure3.csv")
    parser.add_argument('--water_flow_file', type=str, default="D:/GraphormerForRobustness/dataset2/demand_patten_3612/simulate_flow3.csv")
    parser.add_argument('--water_demand_file', type=str, default="D:/GraphormerForRobustness/dataset2/demand_patten_3612/simulate_demand3.csv")

    parser.add_argument('--node_file', type=str, default='D:/GraphormerForRobustness/dataset/Apulia_2/node_features_Apulia2.csv')
    parser.add_argument('--edge_file', type=str, default='D:/GraphormerForRobustness/dataset2/edge_index_dir.csv')
    
    parser.add_argument('--node_adjacency_file', type=str, default='D:/GraphormerForRobustness/dataset/Apulia/node_adjacency_matrix.csv')
    parser.add_argument('--edge_adjacency_file', type=str, default='D:/GraphormerForRobustness/dataset/Apulia/edge_adjacency_matrix.csv')

    parser.add_argument('--incidence_file', type=str, default='D:/GraphormerForRobustness/dataset2/initial_incidence_matrix_dir.csv')
    parser.add_argument('--cycle_file', type=str, default='D:/GraphormerForRobustness/dataset2/initial_cycle_matrix_dir.csv')

    args = parser.parse_args(args=[])  

    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    return args

# Loss Function

In [3]:
# 损失函数 MSE
class ConservationConstraints(nn.Module):
    def __init__(self):
        super(ConservationConstraints, self).__init__()

    def forward(self, pressure_Y, flow_Y, demand_Y, PipeFriction, IncidenceMatrix, CycleMatrix):
        
        # Part 2: Conservation Constraints —— (10.67 Q^1.852L)/(C^1.852 D^4.87)
        PipeFriction = PipeFriction.view(1, 1, -1) 
        pipe_pressure_minus = (abs(flow_Y)**1.852) * PipeFriction 
        pipe_pressure_minus = torch.where(flow_Y > 0, 
                                        pipe_pressure_minus, 
                                        -pipe_pressure_minus)

        loss2_1 = torch.sum((torch.abs((flow_Y @ IncidenceMatrix.t()) + demand_Y)*100)** 2)/96           #  100 -->  targer<0.01 节点流量方程
        loss2_2 = torch.sum((torch.abs((pipe_pressure_minus @ CycleMatrix.t()))*10)** 2)/96              #  10  -->  targer<0.1   环能量方程
        loss2_3 = torch.sum((torch.abs((pressure_Y @ IncidenceMatrix) - pipe_pressure_minus)*10)** 2)/96 #  10  -->  targer<0.1   管段压降方程

        return loss2_1, loss2_2, loss2_3

In [4]:
class ConservationConstraints_generalization(nn.Module):
    def __init__(self):
        super(ConservationConstraints_generalization, self).__init__()
        
    def forward(self, pressure_Y, flow_Y, demand_Y, PipeFriction, IncidenceMatrix, CycleMatrix):
        
        # Part 2: Conservation Constraints —— (10.67 Q^1.852L)/(C^1.852 D^4.87)
        PipeFriction = PipeFriction.view(1,  -1) 
        pipe_pressure_minus = (abs(flow_Y)**1.852) * PipeFriction 
        pipe_pressure_minus = torch.where(flow_Y > 0, 
                                        pipe_pressure_minus, 
                                        -pipe_pressure_minus)
        loss2_1 = torch.sum((torch.abs((flow_Y @ IncidenceMatrix.t()) + demand_Y))** 2, dim=0)/96            #  100 -->  targer<0.01 节点流量方程
        loss2_2 = torch.sum((torch.abs((pipe_pressure_minus @ CycleMatrix.t())))** 2, dim=0)/96              #  10  -->  targer<0.1   环能量方程
        loss2_3 = torch.sum((torch.abs((pressure_Y @ IncidenceMatrix) - pipe_pressure_minus))** 2, dim=0)/96 #  10  -->  targer<0.1   管段压降方程
        
        return loss2_1, loss2_2, loss2_3

# Masking

In [5]:
def apply_mask(args, pressure_data, flow_data, demand_data):
    ''' 
    生成两步掩码：一步掩码张量用于掩盖只剩传感器，二步掩码张量用于继续掩盖传感器
    1. 只有传感器：masked_tensor1 / 掩码方式：mask1
    2. 再对传感器进行掩盖：masked_tensor2 / 掩码方式：mask2
    '''
    pressure_sensor_name = json.loads(args.pressure_sensor_name)
    flow_sensor_name = json.loads(args.flow_sensor_name)
    demand_sensor_name = json.loads(args.demand_sensor_name)

    # 将sensor_index转换为整数类型
    pressure_sensor_index = [int(i)-1 for i in pressure_sensor_name]
    flow_sensor_index = [int(i)-1 for i in flow_sensor_name]
    demand_sensor_index = [int(i)-1 for i in demand_sensor_name]

    # 创建一个和输入张量形状相同的掩码
    mask1_pressure = torch.zeros(pressure_data.shape)
    mask1_flow     = torch.zeros(flow_data.shape)
    mask1_demand   = torch.zeros(pressure_data.shape)
    mask1_pressure[:,pressure_sensor_index] = 1
    mask1_flow[:,flow_sensor_index] = 1
    mask1_demand[:,demand_sensor_index] = 1
    # 去掉mask1_demand的最后一列
    mask1_demand_reduced = mask1_demand[:, :-1]

    mask1_pressure = mask1_pressure.to(args.device)
    mask1_flow = mask1_flow.to(args.device)
    mask1_demand_reduced = mask1_demand_reduced.to(args.device)
    
    return (mask1_pressure, mask1_flow, mask1_demand_reduced)

# DATA

In [6]:
# def seq2instance(args, data):
#     num_step, dims = data.shape
#     # 每增加num_history步 都会产生一个训练样本
#     num_sample = num_step // args.batch_size - 1
#     # x 存储历史时间步数据
#     x = torch.zeros(num_sample, args.batch_size, dims)
#     # y 存储预测数据
#     y = torch.zeros(num_sample, args.batch_size, dims)
#     for i in range(num_sample):
#         j = i * args.num_history
#         x[i] = data[j: j + args.batch_size]
#         y[i] = data[j: j + args.batch_size]
#     # print("x.shape:",x.shape, "y.shape:", y.shape)
#     return x, y

def seq2instance(args, data):
    num_step, dims = data.shape
    # 每增加一步 都会产生一个训练样本
    num_sample = num_step - args.batch_size - args.batch_size + 1
    # x 存储历史时间步数据
    x = torch.zeros(num_sample, args.batch_size, dims)
    # y 存储预测数据
    y = torch.zeros(num_sample, args.batch_size, dims)
    for i in range(num_sample):
        x[i] = data[i: i + args.batch_size]
        y[i] = data[i: i + args.batch_size]
    return x, y

# 将时间序列数据划分为多个训练样本
class CustomDataset(Dataset):
    def __init__(self, args, dataset_type='train'):
        # 读取数据和嵌入
        self.args = args
        self.dataset_type = dataset_type
        
        # 计算训练集、验证集和测试集的时间步数
        train_steps = round(args.train_ratio * args.num_step)
        test_steps = round(args.test_ratio * args.num_step)
        val_steps = args.num_step - train_steps - test_steps

        # 读取节点压力、节点用水量、管道流量数据 
        NodePressure = pd.read_csv(args.water_pressure_file, header=0, index_col=0)
        PipeFlow = pd.read_csv(args.water_flow_file, header=0, index_col=0)
        NodeDemand = pd.read_csv(args.water_demand_file, header=0, index_col=0)

        # min-max 归一化
        NodePressure_ = (NodePressure - NodePressure.min().min()) / (NodePressure.max().max() - NodePressure.min().min())
        PipeFlow_ = (PipeFlow - PipeFlow.min().min()) / (PipeFlow.max().max() - PipeFlow.min().min())
        NodeDemand_ = (NodeDemand - (NodeDemand.iloc[:, :-1]).min().min()) / (NodeDemand.max().max() - (NodeDemand.iloc[:, :-1]).min().min())

        NodePressure = torch.FloatTensor(NodePressure_.values)
        PipeFlow = torch.FloatTensor(PipeFlow_.values)
        NodeDemand = torch.FloatTensor(NodeDemand_.values)

        # 分割数据集
        self.train_pressure = NodePressure[: train_steps]
        self.train_flow     = PipeFlow[: train_steps]
        self.train_demand   = NodeDemand[: train_steps]

        self.val_pressure   = NodePressure[train_steps: train_steps + val_steps]
        self.val_flow       = PipeFlow[train_steps: train_steps + val_steps]
        self.val_demand     = NodeDemand[train_steps: train_steps + val_steps]

        self.test_pressure  = NodePressure[-test_steps:]
        self.test_flow      = PipeFlow[-test_steps:]
        self.test_demand    = NodeDemand[-test_steps:]

        self.trainX_pressure, self.trainY_pressure = seq2instance(args, self.train_pressure)
        self.trainX_flow, self.trainY_flow = seq2instance(args, self.train_flow)
        self.trainX_demand, self.trainY_demand = seq2instance(args, self.train_demand)

        self.valX_pressure, self.valY_pressure = seq2instance(args, self.val_pressure)
        self.valX_flow, self.valY_flow = seq2instance(args, self.val_flow)
        self.valX_demand, self.valY_demand = seq2instance(args, self.val_demand)

        self.testX_pressure, self.testY_pressure = seq2instance(args, self.test_pressure)
        self.testX_flow, self.testY_flow = seq2instance(args, self.test_flow)
        self.testX_demand, self.testY_demand = seq2instance(args, self.test_demand)

    def __len__(self):
        if self.dataset_type == 'train':
            return len(self.trainX_pressure)
        elif self.dataset_type == 'val':
            return len(self.valX_pressure)
        elif self.dataset_type == 'test':
            return len(self.testX_pressure)
        
    def __getitem__(self, idx):
        if self.dataset_type == 'train':
            return (self.trainX_pressure[idx].to(self.args.device), self.trainY_pressure[idx].to(self.args.device),
                    self.trainX_flow[idx].to(self.args.device), self.trainY_flow[idx].to(self.args.device),
                    self.trainX_demand[idx].to(self.args.device), self.trainY_demand[idx].to(self.args.device)
                    )
        
        elif self.dataset_type == 'val':
            return (self.valX_pressure[idx].to(self.args.device), self.valY_pressure[idx].to(self.args.device),
                    self.valX_flow[idx].to(self.args.device), self.valY_flow[idx].to(self.args.device),
                    self.valX_demand[idx].to(self.args.device), self.valY_demand[idx].to(self.args.device)
                    )

        elif self.dataset_type == 'test':
            return (self.testX_pressure[idx].to(self.args.device), self.testY_pressure[idx].to(self.args.device), 
                    self.testX_flow[idx].to(self.args.device), self.testY_flow[idx].to(self.args.device),
                    self.testX_demand[idx].to(self.args.device), self.testY_demand[idx].to(self.args.device)
                    )  

In [7]:
def load_matrix(args):
    # 读取点文件
    nodes_df = pd.read_csv(args.node_file, header=0, index_col=0)
    nodes_attr = torch.from_numpy(nodes_df.values).float().to(args.device)

    # 读取边文件 —— edge weight和海增威廉公式匹配 (10.67*L)/(C^1.852 D^4.87)
    edge_df = pd.read_csv(args.edge_file, header=0, index_col=0)
    edge_index  = edge_df.iloc[:, 0:2].T  # 第0列到第1列
    edge_attr = edge_df.iloc[:, 2:7]      # 第2列到第6列
    edge_index = torch.from_numpy(edge_index.values).long().to(args.device)
    edge_attr = torch.from_numpy(edge_attr.values).float().to(args.device)

    # 生成图数据
    Graph_Data = Data(x=nodes_attr, edge_index=edge_index, edge_attr=edge_attr)

    # 取出edge_df最后一列
    PipeFriction = edge_df.iloc[:, -1].values
    pipe_friction = torch.from_numpy(PipeFriction).float().to(args.device)
    
    # 读取关联矩阵文件
    incidence_matrix_df = pd.read_csv(args.incidence_file, header=None, index_col=None)
    incidence_matrix = torch.from_numpy(incidence_matrix_df.values).float().to(args.device)
    # 读取循环矩阵文件
    cycle_matrix_df = pd.read_csv(args.cycle_file, header=0, index_col=0)
    cycle_matrix = torch.from_numpy(cycle_matrix_df.values).float().to(args.device)

    return Graph_Data, pipe_friction, incidence_matrix, cycle_matrix

# Utiles

In [8]:
class IOStream():
    """训练日志文件"""
    def __init__(self, path):
        self.file = open(path, 'a') # 附加模式：用于在文件末尾添加内容，如果文件不存在则创建新文件

    def cprint(self, text):
        print(text)
        self.file.write(text + '\n')
        self.file.flush() # 确保将写入的内容刷新到文件中，以防止数据在缓冲中滞留

    def close(self):
        self.file.close()

def table_printer(args):
    """绘制参数表格"""
    args = vars(args) # 转成字典类型
    keys = sorted(args.keys()) # 按照字母顺序进行排序
    table = Texttable()
    table.set_cols_dtype(['t', 't']) # 列的类型都为文本(str)
    rows = [["Parameter", "Value"]] # 设置表头
    for k in keys:
        rows.append([k.replace("_", " ").capitalize(), str(args[k])]) # 下划线替换成空格，首字母大写
    table.add_rows(rows)
    return table.draw()

# KAN Parts

In [9]:
# 基于切比雪夫基函数的KAN层
class ChebyshevKANLayer(nn.Module):
    def __init__(self, input_dim, output_dim, degree):
        super(ChebyshevKANLayer, self).__init__()
        self.inputdim = input_dim
        self.outdim = output_dim
        self.ChebyshevDegree = degree
        self.addbias  = True
        self.cheby_coeffs = nn.Parameter(torch.empty(input_dim, output_dim, degree + 1))
        nn.init.normal(self.cheby_coeffs, mean=0.0, std=1/(input_dim * (degree + 1)))
        self.bias = nn.Parameter(torch.zeros(1, output_dim))

    def forward(self, x):
        xshp = x.shape                          # (seq_length, inputdim)
        outshape = xshp[0:-1] + (self.outdim,)  # (seq_length, outdim)
        x = x.view(-1, self.inputdim)           # (seq_length, inputdim)

        x = torch.tanh(x)

        cheby = torch.ones(x.shape[0], self.inputdim, self.ChebyshevDegree + 1, device=x.device)
        if self.ChebyshevDegree > 0:
            cheby[:, :, 1] = x
        for i in range(2, self.ChebyshevDegree + 1):
            cheby[:, :, i] = 2 * x * cheby[:, :, i - 1].clone() - cheby[:, :, i - 2].clone()

        # Compute the Chebyshev interpolation
        # (seq_length, inputdim, degree+1) x (inputdim, output_dim, degree+1) -> (seq_length, output_dim)
        y = torch.einsum('bid,iod->bo', cheby, self.cheby_coeffs)

        # y += self.bias

        y = y.view(outshape) 
        
        return y

In [10]:
# 傅里叶基函数
class FourierKANLayer(nn.Module):
    def __init__(self, input_dim, output_dim, degree):
        super(FourierKANLayer,self).__init__()
        self.gridsize = degree
        self.addbias  = True
        self.inputdim = input_dim
        self.outdim = output_dim

        self.fouriercoeffs = nn.Parameter(
                    torch.randn(2, output_dim, input_dim, degree) / 
                    (np.sqrt(input_dim) * np.sqrt(self.gridsize))
                     )
        nn.init.normal(self.fouriercoeffs, mean=0.0, std=1 / (np.sqrt(input_dim) * np.sqrt(self.gridsize)))
        
        self.bias = nn.Parameter(torch.zeros(1, output_dim))

    def forward(self, x):
        xshp = x.shape                          # (batch_size, seq_length, inputdim)
        outshape = xshp[0:-1] + (self.outdim,)  # (batch_size, seq_lengtha, outdim)
        x = torch.tanh(x)     

        x = x.view(-1, self.inputdim)           # (batch_size*seq_length, inputdim)

        # 创建傅里叶基函数--网格(1, 1, 1, gridsize)
        k = torch.reshape(torch.arange(1, self.gridsize+1, device=x.device), (1, 1, 1, self.gridsize))
        xrshp = x.view(x.shape[0], 1, x.shape[1], 1)   # (batch_size*seq_length, 1, inputdim, 1)

        c = torch.cos(k * xrshp)
        s = torch.sin(k * xrshp)

        y1 = torch.sum(c * self.fouriercoeffs[0:1], (-2, -1)) 
        y2 = torch.sum(s * self.fouriercoeffs[1:2], (-2, -1))

        y = y1 + y2
        
        # y += self.bias

        # 还原(batch_size, seq_length, outdim)
        y = y.view(outshape) 
        
        return y
        

In [11]:
# 注意力机制
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.key_layer   = nn.Linear(hidden_size, hidden_size)
        self.value_layer = nn.Linear(hidden_size, hidden_size)

        self.layer_norm  = nn.LayerNorm(hidden_size)

    def forward(self, attention_from, attention_to):
        # 计算注意力分数
        query = self.query_layer(attention_from)    # (batch_size, hidden_size)
        key = self.key_layer(attention_to)          # (batch_size, hidden_size)
        value = self.value_layer(attention_from)    # (batch_size, hidden_size)

        # 计算注意力权重
        scores = torch.matmul(query, key.T)                # (batch_size, batch_size)
        attention_weights = F.softmax(scores, dim=-1)      # 使用softmax计算归一化权重

        # 计算加权输出
        output = torch.matmul(attention_weights, value)  # (batch_size, hidden_size)
        output = self.layer_norm(output)  # (batch_size, hidden_size)

        return output

# model

In [12]:
class KANSA(nn.Module):
    def __init__(self, args):
        """
        :param num_layers: number of KANSA layers

        :param input_node_dim: input dimension of node features
        :param input_edge_dim: input dimension of edge features

        :param max_path_distance: max pairwise distance between two nodes
        """
        super().__init__()
        self.batch_size = args.batch_size
        self.ChebyshevDegree = args.ChebyshevDegree
        self.FourierDegree = args.FourierDegree

        self.num_nodes = args.num_nodes
        self.num_edges = args.num_edges
        self.hidden_dim = args.hidden_dim

        ''' input ''' 
        # 时间特征
        self.flow_batch = nn.Sequential(
            FourierKANLayer(self.batch_size, self.hidden_dim, self.FourierDegree),  
            FourierKANLayer(self.hidden_dim, self.batch_size, self.FourierDegree),   
            nn.LayerNorm(self.batch_size),            
            )
        self.pressure_batch = nn.Sequential(
            FourierKANLayer(self.batch_size, self.hidden_dim, self.FourierDegree), 
            FourierKANLayer(self.hidden_dim, self.batch_size, self.FourierDegree),   
            nn.LayerNorm(self.batch_size),    
            )
        self.demand_batch = nn.Sequential(
            FourierKANLayer(self.batch_size, self.hidden_dim, self.FourierDegree),   
            FourierKANLayer(self.hidden_dim, self.batch_size, self.FourierDegree),   
            nn.LayerNorm(self.batch_size),      
            )
        # 空间特征
        self.flow_features = nn.Sequential(
            ChebyshevKANLayer(self.num_edges, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.num_edges, self.ChebyshevDegree), 
            nn.LayerNorm(self.num_edges),
            )
        self.pressure_features = nn.Sequential(
            ChebyshevKANLayer(self.num_nodes, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.num_nodes, self.ChebyshevDegree), 
            nn.LayerNorm(self.num_nodes), 
            )
        self.demand_features = nn.Sequential(
            ChebyshevKANLayer(self.num_nodes-1, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.num_nodes-1, self.ChebyshevDegree), 
            nn.LayerNorm(self.num_nodes-1), 
            )
        # 自注意力机制 时间
        self.attention_pressure_batch = Attention(self.batch_size)
        self.attention_flow_batch     = Attention(self.batch_size)
        self.attention_demand_batch   = Attention(self.batch_size)
        # 自注意力机制 空间
        self.attention_pressure_features = Attention(self.num_nodes)
        self.attention_flow_features     = Attention(self.num_edges)
        self.attention_demand_features   = Attention(self.num_nodes-1)

        self.pressure_1 = nn.Sequential(
            nn.LayerNorm(self.num_nodes),
            ChebyshevKANLayer(self.num_nodes, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.hidden_dim, self.ChebyshevDegree)
            )
        self.pressure_2 = nn.Sequential(
            nn.LayerNorm(self.num_nodes),
            ChebyshevKANLayer(self.num_nodes, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.hidden_dim, self.ChebyshevDegree),
            )
        self.flow_1 = nn.Sequential(
            nn.LayerNorm(self.num_edges),
            ChebyshevKANLayer(self.num_edges, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.hidden_dim, self.ChebyshevDegree),
            )
        self.flow_2 = nn.Sequential(
            nn.LayerNorm(self.num_edges),
            ChebyshevKANLayer(self.num_edges, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.hidden_dim, self.ChebyshevDegree),
            )
        self.demand_1 = nn.Sequential(
            nn.LayerNorm(self.num_nodes-1),
            ChebyshevKANLayer(self.num_nodes-1, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.hidden_dim, self.ChebyshevDegree),
            )
        self.demand_2 = nn.Sequential(
            nn.LayerNorm(self.num_nodes-1),
            ChebyshevKANLayer(self.num_nodes-1, self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(self.hidden_dim, self.hidden_dim, self.ChebyshevDegree),
            )

        self.pressure_sum = nn.Sequential(
            nn.LayerNorm(self.hidden_dim),
            ChebyshevKANLayer(self.hidden_dim, 2*self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(2*self.hidden_dim, 2*self.hidden_dim, self.ChebyshevDegree),
            )
        self.flow_sum = nn.Sequential(
            nn.LayerNorm(self.hidden_dim),
            ChebyshevKANLayer(self.hidden_dim, 2*self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(2*self.hidden_dim, 2*self.hidden_dim, self.ChebyshevDegree),
            )
        self.demand_sum = nn.Sequential(
            nn.LayerNorm(self.hidden_dim),
            ChebyshevKANLayer(self.hidden_dim, 2*self.hidden_dim, self.ChebyshevDegree),
            ChebyshevKANLayer(2*self.hidden_dim, 2*self.hidden_dim, self.ChebyshevDegree),
            )

        # 处理器
        self.graph = nn.Sequential(
            nn.LayerNorm(3),
            ChebyshevKANLayer(3, 12, self.ChebyshevDegree),
            ChebyshevKANLayer(12, 1, self.ChebyshevDegree),
            )

        # 图特征解码
        self.flow_out = nn.Sequential(
            ChebyshevKANLayer(2*self.hidden_dim, 2*self.num_edges, self.ChebyshevDegree),
            ChebyshevKANLayer(2*self.num_edges, self.num_edges, self.ChebyshevDegree),
            nn.Sigmoid(),
            )
        self.pressure_out = nn.Sequential(
            ChebyshevKANLayer(2*self.hidden_dim, 2*self.num_nodes, self.ChebyshevDegree),
            ChebyshevKANLayer(2*self.num_nodes, self.num_nodes, self.ChebyshevDegree),
            nn.Sigmoid(),
            )
        # Demand的最后一列是负的（只对前面列进行估计）
        self.demand_out = nn.Sequential(
            ChebyshevKANLayer(2*self.hidden_dim, 2*self.num_nodes, self.ChebyshevDegree),
            ChebyshevKANLayer(2*self.num_nodes, self.num_nodes-1, self.ChebyshevDegree),
            nn.Sigmoid(),
            )

    def forward(self, Graph_Data: Union[Data], 
                mask_pressure, mask_flow, mask_demand,
                pressure, flow, demand) -> Tuple[torch.Tensor, torch.Tensor]:
        # 修改架构，单独注意力
        # 数据预处理
        pressure_ = pressure.float()     # (batch_size, num_nodes)
        flow_ = flow.float()             # (batch_size, num_edges)
        demand_ = demand.float()         # (batch_size, num_nodes)

        # 批次特征提取
        pressure_batch      = self.pressure_batch(pressure_.T)     # (batch_size, num_nodes)
        flow_batch          = self.flow_batch(flow_.T)             # (batch_size, num_edges)
        demand_batch        = self.demand_batch(demand_.T)         # (batch_size, num_nodes)
        pressure_batch      = (self.attention_pressure_batch(pressure_batch, pressure_.T)).T       # (batch_size, num_nodes)
        flow_batch          = (self.attention_flow_batch(flow_batch, flow_.T)).T                   # (batch_size, num_edges)
        demand_batch        = (self.attention_demand_batch(demand_batch, demand_.T)).T             # (batch_size, num_nodes)
        # 节点特征提取
        pressure_features   = self.pressure_features(pressure_)    # (batch_size, num_nodes)
        flow_features       = self.flow_features(flow_)            # (batch_size, num_edges)
        demand_features     = self.demand_features(demand_)        # (batch_size, num_nodes)
        pressure_features   = self.attention_pressure_features(pressure_features, pressure_)       # (batch_size, num_nodes)
        flow_features       = self.attention_flow_features(flow_features, flow_)                   # (batch_size, num_edges)
        demand_features     = self.attention_demand_features(demand_features, demand_)             # (batch_size, num_nodes)
        # 特征线性变换      
        # 结合方式  * is better
        pressure_batch_    = self.pressure_1(pressure_batch * (1-mask_pressure) + pressure_)
        pressure_features_ = self.pressure_2(pressure_features * (1-mask_pressure) + pressure_)
        pressure_st_       = pressure_batch_ * pressure_features_
        
        flow_batch_        = self.flow_1(flow_batch * (1-mask_flow) + flow_)
        flow_features_     = self.flow_2(flow_features * (1-mask_flow) + flow_)
        flow_st_           = flow_batch_ * flow_features_  

        demand_batch_      = self.demand_1(demand_batch * (1-mask_demand)  + demand_)
        demand_features_   = self.demand_2(demand_features * (1-mask_demand) + demand_)
        demand_st_         = demand_batch_ * demand_features_

        # 拼接
        Graph_features = torch.cat((self.pressure_sum(pressure_st_).unsqueeze(-1), 
                                    self.flow_sum(flow_st_).unsqueeze(-1), 
                                    self.demand_sum(demand_st_).unsqueeze(-1)), 
                                    dim=-1)
        graph_features = self.graph(Graph_features).squeeze(-1)

        # 解码
        pressure_dd = self.pressure_out(graph_features) * (1-mask_pressure)   + pressure_ 
        flow_dd = self.flow_out(graph_features)         * (1-mask_flow)       + flow_ 
        demand_dd = self.demand_out(graph_features)     * (1-mask_demand)     + demand_

        return pressure_dd, flow_dd, demand_dd

# Main

In [13]:
def train(args, IO,  train_loader, val_loader, 
        min_pressure, max_pressure, min_flow, max_flow, min_demand, max_demand):
    best_val_loss = float('inf')
    patience = args.patience  
    patience_counter = 0

    # 使用GPU or CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.manual_seed(args.seed)  
    
    # 加载需求数据
    Graph_Data, pipe_friction, incidence_matrix, cycle_matrix = load_matrix(args)

    # 加载模型及参数量统计
    model = KANSA(args).to(device)

    IO.cprint(str(model))
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    IO.cprint('Model Parameter: {}'.format(total_params))
    
    # 优化器 RMSprop  Trick
    optimizer = optim.RMSprop(model.parameters(), lr=args.learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                        step_size=args.decay_epoch,
                                        gamma=args.gamma)
    IO.cprint('Using RMSprop')

    # 损失函数
    criterion = ConservationConstraints()
    # 保存损失
    train_loss1_list = []
    train_loss2_list = []
    train_loss3_list = []
    val_loss1_list = []
    val_loss2_list = []
    val_loss3_list = []
    start_time = time.time()
    for epoch in range(args.max_epoch):
        #################
        ###   Train   ###
        #################
        model.train()  
        train_loss1 = 0.0 
        train_loss2 = 0.0
        train_loss3 = 0.0
        pressure_True = torch.Tensor().to(args.device)
        pressure_Est  = torch.Tensor().to(args.device)
        flow_True     = torch.Tensor().to(args.device)
        flow_Est      = torch.Tensor().to(args.device)
        demand_True   = torch.Tensor().to(args.device)
        demand_Est    = torch.Tensor().to(args.device)

        for i, data in tqdm(enumerate(train_loader), total=len(train_loader), desc="Train_Loader"):
            # 加载数据
            (train_pressureX, train_pressureY, 
            train_flowX, train_flowY, 
            train_demandX, train_demandY) = data
            train_pressureX = train_pressureX.squeeze(0)
            train_flowX     = train_flowX.squeeze(0)
            train_demandX   = train_demandX.squeeze(0)

            # 随机掩码
            (mask1_pressure, mask1_flow, mask1_demand)=apply_mask(args, train_pressureX, train_flowX, train_demandX)

            # 掩码
            train_pressureX_ = train_pressureX * mask1_pressure
            train_flowX_     = train_flowX * mask1_flow
            train_demandX_   = (train_demandX[:, :-1]) * mask1_demand

            # model
            pressure_Y, flow_Y, demand_Y = model(Graph_Data, mask1_pressure, mask1_flow, mask1_demand, 
                                                train_pressureX_, train_flowX_, train_demandX_)
            # 给demand_Y添加最后一列
            row_sums = torch.sum(demand_Y, dim=1, keepdim=True)
            demand_Y = torch.cat((demand_Y, -row_sums), dim=1)

            # min-max归一化还原
            pressure_Y = (pressure_Y * (max_pressure - min_pressure) + min_pressure)
            flow_Y     = (flow_Y     * (max_flow     - min_flow)     + min_flow)
            demand_Y   = (demand_Y   * (max_demand   - min_demand)   + min_demand)
            train_pressureX  = (train_pressureX  * (max_pressure - min_pressure) + min_pressure)
            train_pressureX_ = (train_pressureX_ * (max_pressure - min_pressure) + min_pressure)
            train_flowX      = (train_flowX      * (max_flow     - min_flow)     + min_flow)
            train_flowX_     = (train_flowX_     * (max_flow     - min_flow)     + min_flow)
            train_demandX    = (train_demandX    * (max_demand   - min_demand)   + min_demand)
            train_demandX_   = (train_demandX_   * (max_demand   - min_demand)   + min_demand)
            
            # 计算损失
            (loss1, loss2, loss3) = criterion(pressure_Y, flow_Y, demand_Y, pipe_friction, incidence_matrix, cycle_matrix)

            # 数据保存
            pressure_True = torch.cat((pressure_True, train_pressureX), 0)
            pressure_Est  = torch.cat((pressure_Est, pressure_Y), 0)
            flow_True     = torch.cat((flow_True,   train_flowX), 0)
            flow_Est      = torch.cat((flow_Est,     flow_Y), 0)
            demand_True   = torch.cat((demand_True, train_demandX), 0)
            demand_Est    = torch.cat((demand_Est,   demand_Y), 0)

            # l1+l2正则化
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            loss_ = loss1 + loss2 + loss3

            loss = loss_ + args.l1_lambda * l1_norm + args.l2_lambda * l2_norm
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            train_loss1 += (loss1).item()
            train_loss2 += (loss2).item()
            train_loss3 += (loss3).item()

        scheduler.step()
        avg_train_loss1 = (train_loss1) / len(train_loader.dataset)
        avg_train_loss2 = (train_loss2) / len(train_loader.dataset)
        avg_train_loss3 = (train_loss3) / len(train_loader.dataset)
        # 绘图保存
        train_loss1_list.append(avg_train_loss1)
        train_loss2_list.append(avg_train_loss2)
        train_loss3_list.append(avg_train_loss3)

        IO.cprint('Epoch #{:03d}, Conservation1: {:.4f}, Conservation2: {:.4f}, Conservation3: {:.4f}'.format(
                    epoch, (avg_train_loss1), (avg_train_loss2), (avg_train_loss3)
                    ))

        #################
        ###   Valid   ###
        #################
        model.eval()
        val_loss1 = 0.0
        val_loss2 = 0.0
        val_loss3 = 0.0
        with torch.no_grad():
            for i, data in tqdm(enumerate(val_loader), total=len(val_loader), desc="Val_Loader"):

                (val_pressureX, val_pressureY, 
                 val_flowX, val_flowY,
                 val_demandX, val_demandY)= data
                val_pressureX = val_pressureX.squeeze(0)
                val_flowX     = val_flowX.squeeze(0)
                val_demandX   = val_demandX.squeeze(0)

                # 随机掩码
                (mask1_pressure, mask1_flow, mask1_demand)=apply_mask(args, val_pressureX, val_flowX, val_demandX)
                val_pressureX_ = val_pressureX * mask1_pressure
                val_flowX_     = val_flowX * mask1_flow
                val_demandX_   = (val_demandX[:, :-1]) * mask1_demand

                pressure_Y, flow_Y, demand_Y = model(Graph_Data, mask1_pressure, mask1_flow, mask1_demand, val_pressureX_, val_flowX_, val_demandX_)
                # 给demand_Y添加最后一列
                row_sums = torch.sum(demand_Y, dim=1, keepdim=True)
                demand_Y = torch.cat((demand_Y, -row_sums), dim=1)
                
                # min-max归一化还原
                pressure_Y = (pressure_Y * (max_pressure - min_pressure) + min_pressure)
                flow_Y     = (flow_Y     * (max_flow     - min_flow)     + min_flow)
                demand_Y   = (demand_Y   * (max_demand   - min_demand)   + min_demand)
                val_pressureX  = (val_pressureX * (max_pressure - min_pressure) + min_pressure)
                val_flowX      = (val_flowX     * (max_flow     - min_flow)     + min_flow)
                val_demandX    = (val_demandX   * (max_demand   - min_demand)   + min_demand)
                val_pressureX_ = (val_pressureX_ * (max_pressure- min_pressure) + min_pressure)
                val_flowX_     = (val_flowX_     * (max_flow    - min_flow)     + min_flow)
                val_demandX_   = (val_demandX_   * (max_demand   - min_demand)  + min_demand)

                # 计算损失
                (loss1, loss2, loss3) = criterion(pressure_Y, flow_Y, demand_Y, pipe_friction, incidence_matrix, cycle_matrix)
                
                # 数据保存
                pressure_True = torch.cat((pressure_True, val_pressureX), 0)
                pressure_Est  = torch.cat((pressure_Est, pressure_Y), 0)
                flow_True     = torch.cat((flow_True,   val_flowX), 0)
                flow_Est      = torch.cat((flow_Est,     flow_Y), 0)
                demand_True   = torch.cat((demand_True, val_demandX), 0)
                demand_Est    = torch.cat((demand_Est,   demand_Y), 0)

                # 误差计算
                val_loss1 += loss1.item()
                val_loss2 += loss2.item()
                val_loss3 += loss3.item()
        avg_val_loss1 = (val_loss1) / len(val_loader.dataset)
        avg_val_loss2 = (val_loss2) / len(val_loader.dataset)
        avg_val_loss3 = (val_loss3) / len(val_loader.dataset)
        avg_val_loss = (avg_val_loss1 + avg_val_loss2 + avg_val_loss3)
        # 绘图保存
        val_loss1_list.append(avg_val_loss1)
        val_loss2_list.append(avg_val_loss2)
        val_loss3_list.append(avg_val_loss3)
        
        IO.cprint('Epoch #{:03d}, Conservation1: {:.4f}, Conservation2: {:.4f}, Conservation3: {:.4f}'.format(
                epoch, (avg_val_loss1), (avg_val_loss2), (avg_val_loss3)
                ))

        # 选取最佳参数
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # 保存当前最佳模型
            best_model_wts = model.state_dict()
        else:
            patience_counter += 1

        # 检查是否达到提前停止条件
        if patience_counter >= patience:
            IO.cprint('Early stopping triggered. Best Val_Loss: {:.4f}'.format(best_val_loss))
            break
        
    model.load_state_dict(best_model_wts)

    torch.save(model, 'outputs/%s/model.pth' % args.exp_name)
    IO.cprint('The current best model is saved in: {}'.format('******** outputs/%s/model.pth *********' % args.exp_name))
    end_time = time.time()
    IO.cprint('Total time: {:.4f}s'.format(end_time - start_time))
    return (pressure_True, pressure_Est, flow_True, flow_Est, demand_True, demand_Est,
            train_loss1_list, train_loss2_list, train_loss3_list, 
            val_loss1_list, val_loss2_list, val_loss3_list)

In [14]:
def test(args, IO, test_loader, 
        min_pressure, max_pressure, min_flow, max_flow, min_demand, max_demand):
    """测试模型"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 加载需求数据
    Graph_Data, pipe_friction, incidence_matrix, cycle_matrix = load_matrix(args)
    
    # 输出内容保存在之前的训练日志里
    IO.cprint('')
    IO.cprint('********** TEST START **********')
    IO.cprint('Reload Best Model')
    IO.cprint('The current best model is saved in: {}'.format('******** outputs/%s/model.pth *********' % args.exp_name))

    model = torch.load('outputs/%s/model.pth' % args.exp_name).to(device)
    model = model.train()
    
    optimizer = optim.RMSprop(model.parameters(), lr=1e-7)
                                        
    ##############################
    ### Test For Generalization ##
    ##############################

    # 损失函数
    criterion1 = ConservationConstraints()
    criterion2 = ConservationConstraints_generalization()

    # 加载需求数据
    offset = 0  # 初始偏移量
    batch_size = 3 * 96

    MAE_pressure = []
    MAE_flow = []
    MAE_demand = []
    MAPE_pressure = []
    MAPE_flow = []
    MAPE_demand = []
    R2_pressure = []
    R2_flow = []
    R2_demand = []
    Loss1 = []
    Loss2 = []
    Loss3 = []

    for loop_ in range(len(test_loader) - 3 * 96):

        start_index = offset
        end_index = min(start_index + batch_size, len(test_loader))
        sliced_dataset = torch.utils.data.Subset(test_loader.dataset, range(start_index, end_index))
        new_test_loader = torch.utils.data.DataLoader(sliced_dataset, batch_size=1, shuffle=False)

        for i, data in tqdm(enumerate(new_test_loader), total=len(new_test_loader), desc="Test_Loader"):
            (test_pressureX, test_pressureY,
            test_flowX, test_flowY, 
            test_demandX, test_demandY)= data
            test_pressureX = test_pressureX.squeeze(0)
            test_flowX     = test_flowX.squeeze(0)
            test_demandX   = test_demandX.squeeze(0)

            # 随机掩码
            (mask1_pressure, mask1_flow, mask1_demand)=apply_mask(args, test_pressureX, test_flowX, test_demandX)

            # 掩码
            test_pressureX_ = test_pressureX * mask1_pressure
            test_flowX_     = test_flowX * mask1_flow
            test_demandX_   = (test_demandX[:, :-1]) * mask1_demand

            pressure_Y, flow_Y, demand_Y = model(Graph_Data, mask1_pressure, mask1_flow, mask1_demand, test_pressureX_, test_flowX_ , test_demandX_)
            # 给demand_Y添加最后一列
            row_sums = torch.sum(demand_Y, dim=1, keepdim=True)
            demand_Y = torch.cat((demand_Y, -row_sums), dim=1)
        
            # min-max归一化还原
            pressure_Y = (pressure_Y * (max_pressure - min_pressure) + min_pressure)
            flow_Y     = (flow_Y     * (max_flow     - min_flow)     + min_flow)
            demand_Y   = (demand_Y   * (max_demand   - min_demand)   + min_demand)
            test_pressureX = (test_pressureX * (max_pressure - min_pressure) + min_pressure)
            test_flowX     = (test_flowX     * (max_flow     - min_flow)     + min_flow)
            test_demandX   = (test_demandX   * (max_demand   - min_demand)   + min_demand)

            (loss1, loss2, loss3) = criterion1(pressure_Y, flow_Y, demand_Y, pipe_friction, incidence_matrix, cycle_matrix)
            loss_ = loss1 + loss2 + loss3
            loss_.backward()
            optimizer.step()
            optimizer.zero_grad()

            # 运行的最后一个批次进行保存
            if i == len(new_test_loader) - 1:

                IO.cprint('Loss1: {:.4f}, Loss2: {:.4f}, Loss3: {:.4f}'.format(loss1.item(), loss2.item(), loss3.item()))
                (loss1, loss2, loss3) = criterion2(pressure_Y, flow_Y, demand_Y, pipe_friction, incidence_matrix, cycle_matrix)

                # 数据处理
                mae_pressure = torch.mean(torch.abs(pressure_Y - test_pressureX), dim=0)
                mae_flow     = torch.mean(torch.abs(flow_Y - test_flowX), dim=0)
                mae_demand   = torch.mean(torch.abs(demand_Y - test_demandX), dim=0)

                mape_pressure = torch.mean(torch.abs((pressure_Y - test_pressureX) / test_pressureX), dim=0)
                mape_flow     = torch.mean(torch.abs((flow_Y - test_flowX) / test_flowX), dim=0)
                mape_demand   = torch.mean(torch.abs((demand_Y - test_demandX) / test_demandX), dim=0)

                r2_pressure = 1 - (torch.sum(((pressure_Y - test_pressureX) ** 2), dim=0) / torch.sum((pressure_Y - torch.mean(pressure_Y)) ** 2, dim=0))
                r2_flow     = 1 - (torch.sum(((flow_Y - test_flowX) ** 2), dim=0) / torch.sum((flow_Y - torch.mean(flow_Y)) ** 2, dim=0))
                r2_demand   = 1 - (torch.sum(((demand_Y - test_demandX) ** 2), dim=0) / torch.sum((demand_Y - torch.mean(demand_Y)) ** 2, dim=0))

                Loss1.append(loss1.detach().cpu().numpy())
                Loss2.append(loss2.detach().cpu().numpy())
                Loss3.append(loss3.detach().cpu().numpy())

                MAE_pressure.append(mae_pressure.detach().cpu().numpy())
                MAE_flow.append(mae_flow.detach().cpu().numpy())
                MAE_demand.append(mae_demand.detach().cpu().numpy())

                MAPE_pressure.append(mape_pressure.detach().cpu().numpy())
                MAPE_flow.append(mape_flow.detach().cpu().numpy())
                MAPE_demand.append(mape_demand.detach().cpu().numpy())

                R2_pressure.append(r2_pressure.detach().cpu().numpy())
                R2_flow.append(r2_flow.detach().cpu().numpy())  
                R2_demand.append(r2_demand.detach().cpu().numpy())

                offset += 1

            del pressure_Y, flow_Y, demand_Y, loss1, loss2, loss3
            del test_pressureX, test_flowX, test_demandX, test_pressureX_, test_flowX_, test_demandX_

    return (Loss1, Loss2, Loss3,
            MAE_pressure, MAE_flow, MAE_demand, 
            MAPE_pressure, MAPE_flow, MAPE_demand,
            R2_pressure, R2_flow, R2_demand)

In [15]:
args = parse_args()
def exp_init():
    """实验初始化"""
    if not os.path.exists('outputs'):
        os.mkdir('outputs')
    if not os.path.exists('outputs/' + args.exp_name):
        os.mkdir('outputs/' + args.exp_name)

In [16]:
if __name__ == '__main__':
      random.seed(args.seed)  # 设置Python随机种子
      torch.manual_seed(args.seed)  # 设置PyTorch随机种子
      exp_init()      
      IO = IOStream('outputs/' + args.exp_name + '/run.log')
      IO.cprint(str(table_printer(args)))  

      train_dataset = CustomDataset(args, dataset_type='train')
      val_dataset = CustomDataset(args, dataset_type='val')
      test_dataset = CustomDataset(args, dataset_type='test')
      train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, drop_last=True)
      val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=True)
      test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=True)

      # 读取节点压力、节点用水量、管道流量数据 
      NodePressure = pd.read_csv(args.water_pressure_file, header=0, index_col=0)
      PipeFlow = pd.read_csv(args.water_flow_file, header=0, index_col=0)
      NodeDemand = pd.read_csv(args.water_demand_file, header=0, index_col=0)
      max_pressure = NodePressure.max().max()
      min_pressure = NodePressure.min().min()
      # 不要乘以1000，要按m3/s进行计算
      max_flow = PipeFlow.max().max() 
      min_flow = PipeFlow.min().min()
      max_demand = NodeDemand.max().max()
      min_demand = (NodeDemand.iloc[:, :-1]).min().min()                          
      # TODO 这里要是0，最小值在水库取得为负。
      
      # 转换为PyTorch张量并移动到CUDA设备
      min_pressure = torch.tensor(min_pressure).to(args.device)
      max_pressure = torch.tensor(max_pressure).to(args.device)
      min_flow = torch.tensor(min_flow).to(args.device)
      max_flow = torch.tensor(max_flow).to(args.device)

      (pressure_True, pressure_Est, flow_True, flow_Est, demand_True, demand_Est,
       train_loss1_list, train_loss2_list, train_loss3_list,
       val_loss1_list, val_loss2_list, val_loss3_list) = train(
       args, IO, train_dataloader, val_dataloader, min_pressure, max_pressure, min_flow, max_flow, min_demand, max_demand)
      
      (Loss1, Loss2, Loss3,
       MAE_pressure, MAE_flow, MAE_demand, 
       MAPE_pressure, MAPE_flow, MAPE_demand,
       R2_pressure, R2_flow, R2_demand) = test(
      args, IO, test_dataloader, min_pressure, max_pressure, min_flow, max_flow, min_demand, max_demand) 

+----------------------+-------------------------------------------------------+
|      Parameter       |                         Value                         |
+======================+=======================================================+
| Chebyshevdegree      | 8                                                     |
+----------------------+-------------------------------------------------------+
| Fourierdegree        | 6                                                     |
+----------------------+-------------------------------------------------------+
| Batch size           | 96                                                    |
+----------------------+-------------------------------------------------------+
| Cycle file           | D:/GraphormerForRobustness/dataset2/initial_cycle_mat |
|                      | rix_dir.csv                                           |
+----------------------+-------------------------------------------------------+
| Decay epoch          | 1  

C:\AppData\Local\Temp\ipykernel_10972\2426456413.py:14: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.fouriercoeffs, mean=0.0, std=1 / (np.sqrt(input_dim) * np.sqrt(self.gridsize)))
C:\AppData\Local\Temp\ipykernel_10972\1574067456.py:10: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.cheby_coeffs, mean=0.0, std=1/(input_dim * (degree + 1)))


KANSA(
  (flow_batch): Sequential(
    (0): FourierKANLayer()
    (1): FourierKANLayer()
    (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pressure_batch): Sequential(
    (0): FourierKANLayer()
    (1): FourierKANLayer()
    (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (demand_batch): Sequential(
    (0): FourierKANLayer()
    (1): FourierKANLayer()
    (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (flow_features): Sequential(
    (0): ChebyshevKANLayer()
    (1): ChebyshevKANLayer()
    (2): LayerNorm((34,), eps=1e-05, elementwise_affine=True)
  )
  (pressure_features): Sequential(
    (0): ChebyshevKANLayer()
    (1): ChebyshevKANLayer()
    (2): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
  )
  (demand_features): Sequential(
    (0): ChebyshevKANLayer()
    (1): ChebyshevKANLayer()
    (2): LayerNorm((23,), eps=1e-05, elementwise_affine=True)
  )
  (attention_pressure_batch): Attention(
    (query_layer): Linear(in_fe

Train_Loader: 100%|██████████| 4129/4129 [08:16<00:00,  8.31it/s]


Epoch #000, Conservation1: 219321.8637, Conservation2: 37884443.3502, Conservation3: 50439025.6250


...

Loss1: 0.3276, Loss2: 0.1886, Loss3: 0.5322


Test_Loader: 100%|██████████| 288/288 [00:28<00:00, 10.23it/s]


Loss1: 0.3183, Loss2: 0.1841, Loss3: 0.5413


Test_Loader: 100%|██████████| 288/288 [00:27<00:00, 10.34it/s]

Loss1: 0.3236, Loss2: 0.1812, Loss3: 0.5434


In [17]:
# 创建输出目录
output_dir = 'outputs1/DataManage'  
os.makedirs(output_dir, exist_ok=True)  

In [18]:
# # 数据输出到csv文件 
# save_pressure_True = pressure_True.cpu().detach().numpy().astype(float)
# save_pressure_Est = pressure_Est.cpu().detach().numpy().astype(float)
# save_flow_True = flow_True.cpu().detach().numpy().astype(float)
# save_flow_Est = flow_Est.cpu().detach().numpy().astype(float)
# save_demand_True = demand_True.cpu().detach().numpy().astype(float)
# save_demand_Est = demand_Est.cpu().detach().numpy().astype(float)

# np.savetxt(os.path.join(output_dir, 'pressure_True.csv'), save_pressure_True, delimiter=',')
# np.savetxt(os.path.join(output_dir, 'pressure_Est.csv'), save_pressure_Est, delimiter=',')
# np.savetxt(os.path.join(output_dir, 'flow_True.csv'), save_flow_True, delimiter=',')  
# np.savetxt(os.path.join(output_dir, 'flow_Est.csv'), save_flow_Est, delimiter=',')
# np.savetxt(os.path.join(output_dir, 'demand_True.csv'), save_demand_True, delimiter=',')
# np.savetxt(os.path.join(output_dir, 'demand_Est.csv'), save_demand_Est, delimiter=',')

In [19]:
# 泛化保存 (MAE_pressure, MAE_flow, MAE_demand, MAPE_pressure, MAPE_flow, MAPE_demand, R2_pressure, R2_flow, R2_demand) 
MAE_pressure = pd.DataFrame(MAE_pressure)
MAE_flow = pd.DataFrame(MAE_flow)
MAE_demand = pd.DataFrame(MAE_demand)

MAPE_pressure = pd.DataFrame(MAPE_pressure)
MAPE_flow = pd.DataFrame(MAPE_flow)
MAPE_demand = pd.DataFrame(MAPE_demand)

R2_pressure = pd.DataFrame(R2_pressure)
R2_flow = pd.DataFrame(R2_flow)
R2_demand = pd.DataFrame(R2_demand)

# 保存结果
Loss1 = pd.DataFrame(Loss1)
Loss2 = pd.DataFrame(Loss2)
Loss3 = pd.DataFrame(Loss3)
Loss1.to_csv(os.path.join(output_dir, 'Coservation1_generalization.csv'), index=False)
Loss2.to_csv(os.path.join(output_dir, 'Coservation2_generalization.csv'), index=False)
Loss3.to_csv(os.path.join(output_dir, 'Coservation3_generalization.csv'), index=False)

MAE_pressure.to_csv(os.path.join(output_dir, 'MAE_pressure_generalization.csv'), index=False)
MAE_flow.to_csv(os.path.join(output_dir, 'MAE_flow_generalization.csv'), index=False)
MAE_demand.to_csv(os.path.join(output_dir, 'MAE_demand_generalization.csv'), index=False)

MAPE_pressure.to_csv(os.path.join(output_dir, 'MAPE_pressure_generalization.csv'), index=False)
MAPE_flow.to_csv(os.path.join(output_dir, 'MAPE_flow_generalization.csv'), index=False)
MAPE_demand.to_csv(os.path.join(output_dir, 'MAPE_demand_generalization.csv'), index=False)

R2_pressure.to_csv(os.path.join(output_dir, 'R2_pressure_generalization.csv'), index=False)
R2_flow.to_csv(os.path.join(output_dir, 'R2_flow_generalization.csv'), index=False)
R2_demand.to_csv(os.path.join(output_dir, 'R2_demand_generalization.csv'), index=False)